In [25]:
from pathlib import Path
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm

In [26]:
def is_white_layout(
    image: Image.Image,
    brightness_threshold: int = 230,
    white_pixel_ratio: float = 0.9,
) -> bool:
    """
    Проверяет, является ли изображение "белой" планировкой квартиры.
    
    Параметры:
    - image: изображение в формате PIL.Image
    - brightness_threshold: порог яркости для "белого" пикселя (0-255)
    - white_pixel_ratio: минимальная доля белых пикселей
    
    Возвращает:
    - True, если изображение соответствует критериям "белой" планировки
    """
    img_array = np.array(image.convert("L"))  # Конвертируем в grayscale и NumPy-массив
    
    mean_brightness = np.mean(img_array)
    white_pixels = np.sum(img_array >= brightness_threshold)
    white_ratio = white_pixels / img_array.size
    
    return (mean_brightness >= brightness_threshold) or (white_ratio >= white_pixel_ratio)

In [27]:
def filter_white_layouts(
    data_dir: str | Path,
    output_dir: str | Path,
    brightness_threshold: int = 230,
    white_pixel_ratio: float = 0.9,
) -> int:
    """
    Фильтрует изображения в папке `data_dir` и сохраняет "белые" планировки в `output_dir`.
    
    Параметры:
    - data_dir: путь к папке с изображениями (str или Path)
    - output_dir: путь к папке для сохранения (str или Path)
    - brightness_threshold, white_pixel_ratio: параметры фильтрации
    """
    data_path = Path(data_dir)
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True, parents=True)

    move_counter = 0
    # Получаем список всех изображений заранее для прогресс-бара
    image_paths = [p for p in data_path.glob("*.*") if p.suffix.lower() in ('.png', '.jpg', '.jpeg')]
    for img_path in tqdm(image_paths, desc="Обработка изображений"):
        if img_path.suffix.lower() not in ('.png', '.jpg', '.jpeg'):
            continue
        
        try:
            with Image.open(img_path) as img:
                if is_white_layout(img, brightness_threshold, white_pixel_ratio):
                    shutil.move(img_path, output_dir)
                    move_counter += 1
        except Exception as e:
            print(f"Ошибка при обработке {img_path.name}: {e}")
    return move_counter

In [29]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
cian_data_dir = data_dir / "cian_data"
output_data_dir = data_dir / "floor_plans"

for dirname in cian_data_dir.iterdir():
    print(dirname)
    move_counter = filter_white_layouts(
        data_dir=dirname,
        output_dir=output_data_dir,
        brightness_threshold=220,
        white_pixel_ratio=0.86
    )
    print(f"Перемещено файлов: {move_counter}")

Директория: /home/little-garden/CodeProjects/InteriorClass
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/A0_uncertain


Обработка изображений: 100%|██████████| 37820/37820 [00:31<00:00, 1187.84it/s]


Перемещено файлов: 253
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/D0


Обработка изображений: 100%|██████████| 114399/114399 [01:28<00:00, 1299.71it/s]


Перемещено файлов: 198
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/A1


...

Перемещено файлов: 1
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/D1


Обработка изображений: 100%|██████████| 99370/99370 [01:16<00:00, 1300.67it/s]


Перемещено файлов: 119
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/A0


Обработка изображений: 100%|██████████| 43212/43212 [00:33<00:00, 1279.29it/s]

Перемещено файлов: 251


In [19]:
image = Image.open("/home/little-garden/CodeProjects/InteriorClass/data/cian_data/A0/kvartira-moskva-minskaya-ulica-2405225048-4.jpg")
is_white_layout(image)

230 0.9


np.False_

In [ ]:
# Пример использования
data_dir = "data_dir"  # Папка с исходными изображениями (можно передать как Path)
output_dir = "white_layouts"  # Папка для "белых" планировок
filter_white_layouts(data_dir, output_dir)